# Neural Network for Predicting World Series Champions
The procedure has been borrowed in part from the machine learning class examples

# Read in data and pre-process

In [1]:
# import dependencies
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
# read in data
pd.set_option('display.max_columns', None)
mlb_df = pd.read_csv('resources/mlb_data.csv')
mlb_df.head()

,Year-Tm,Year,Tm,Lg,W-L%,R/G-hit,HR-hit,RBI-hit,SB-hit,BB-hit,SO-hit,BA-hit,OBP-hit,SLG-hit,OPS-hit,LOB-hit,OPS+-hit,ERA-pitch,WHIP-pitch,H9-pitch,HR9-pitch,SO/W-pitch,ERA+-pitch,Winner
0,2019MIN,2019,MIN,AL,0.623,5.80,307,906,28,525,1334,0.270,0.338,0.494,0.832,1115,119,4.18,1.304,9.0,1.2,3.24,108,False
1,2019NYY,2019,NYY,AL,0.636,5.82,306,904,55,569,1437,0.267,0.339,0.490,0.829,1039,117,4.31,1.304,8.6,1.5,3.03,106,False
2,2019HOU,2019,HOU,AL,0.660,5.68,288,891,67,645,1166,0.274,0.352,0.495,0.848,1168,120,3.66,1.130,7.4,1.4,3.73,127,False
3,2019LAD,2019,LAD,NL,0.654,5.47,279,861,57,607,1356,0.257,0.338,0.472,0.810,1124,112,3.39,1.102,7.5,1.2,3.88,123,False
4,2018NYY,2018,NYY,AL,0.617,5.25,267,821,63,625,1421,0.249,0.329,0.451,0.781,1100,112,3.78,1.239,8.1,1.1,3.31,112,False


In [3]:
# change the League column to be binary instead of string
for i, row in mlb_df.iterrows():
#     print(i,row['Lg'])
    if row['Lg'] == 'AL':
        mlb_df.at[i,'Lg'] = 0
#         print(f'new value at {i} is {mlb_df.at[i,"Lg"]}')
    elif row['Lg'] == 'NL':
        mlb_df.at[i,'Lg'] = 1
#         print(f'new value at {i} is {mlb_df.at[i,"Lg"]}')
    

In [92]:
# league is now a 1 or 0
mlb_df.head()
mlb_df.to_html('raw.html')

# Split into training and testing set
Manually split the data by season. Odd years will be used for training, while even years will be used for testing

In [5]:
# get X and Y train for making the model
train_df = mlb_df.loc[mlb_df['Year'] % 2 == 1]
train_data = train_df.values
X_train = train_data[:, 3:22]
y_train = train_data[:,23]

In [6]:
# get X and Y for making the model
test_df = mlb_df.loc[mlb_df['Year'] % 2 == 0]
test_data = test_df.values
X_test = test_data[:, 3:22]
y_test = test_data[:,23]

# Scaling and One-hot encoding

In [7]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

In [8]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# label encode the winner column
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
label_encoder.fit(y_test)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# One-hot encoding
from keras.utils import to_categorical

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
# y_train_categorical

Using TensorFlow backend.


# Create model

In [11]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [12]:
X_train.shape

(300, 19)

In [13]:
from tensorflow.keras.layers import Dense
number_inputs = X_train.shape[1]
number_hidden_nodes = 1000
model.add(Dense(units=number_hidden_nodes, activation='relu', input_dim=number_inputs))
model.add(Dense(units=number_hidden_nodes, activation='relu'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000)              20000     
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2002      
Total params: 1,023,002
Trainable params: 1,023,002
Non-trainable params: 0
_________________________________________________________________


In [16]:
# # Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [17]:
print(X_train_scaled.shape)
print(y_train_categorical.shape)

(300, 19)
(300, 2)


In [18]:
# Fit the model
model.fit(
    x=X_train_scaled,
    y=y_train_categorical,
    epochs=200,
    shuffle=True,
    verbose=0
)

In [19]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

330/330 - 0s - loss: 0.8965 - acc: 0.9485
Loss: 0.896494035616122, Accuracy: 0.9484848380088806


In [20]:
encoded_predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
# print(f"Predicted classes: {prediction_labels}")
# print(f"Actual Labels: {list(y_test)}")

# Post-process and analyze results

In [21]:
pd.set_option('display.max_rows', None)
predictions_df = pd.DataFrame({'Prediction':prediction_labels, 'Actual':list(y_test)})
# predictions_df.head()

In [22]:
winners_df = predictions_df.loc[predictions_df['Actual']]
# winners_df

In [23]:
# connect predictions to the team and season using iloc and test_df
teams = []
years = []
for i, row in winners_df.iterrows():
    team = test_df.iloc[i,2]
    year = test_df.iloc[i,1]
    teams.append(team)
    years.append(year)

In [24]:
# display predictions by team
winners_df['Team'] = teams
winners_df['Year'] = years
# winners_df

In [25]:
# using probabilities 
probs = model.predict_proba(X_test_scaled)
# probs

In [26]:
# dataframe cleanup
probs_df = pd.DataFrame.from_records(probs)
probs_df[0] = probs_df[0] * 100
probs_df[1] = probs_df[1] * 100
probs_df.rename(columns={0:'False', 1:'True'}, inplace=True)
probs_df.head()

,False,True
0,100.000000,5.018482e-23
1,100.000000,6.556704e-23
2,100.000000,6.546619e-27
3,100.000000,3.699494e-30
4,99.999869,1.271206e-04


In [27]:
# connect predictions to the team and season using iloc and test_df
all_teams = []
all_years = []
for i, row in probs_df.iterrows():
    team = test_df.iloc[i,2]
    year = test_df.iloc[i,1]
    all_teams.append(team)
    all_years.append(year)
# print(all_years)

In [28]:
# add teams to dataframe
probs_df['Year'] = all_years
probs_df['Team'] = all_teams
probs_df.sort_values(by=['Year','True'],ascending=False)
# probs_df.head()

,False,True,Year,Team
285,0.000524,9.999948e+01,2020,LAD
301,0.018476,9.998153e+01,2020,SDP
313,93.851602,6.148403e+00,2020,TBR
300,99.980897,1.910529e-02,2020,CHW
305,99.991369,8.626504e-03,2020,MIN
297,99.999988,7.841968e-06,2020,ATL
312,100.000000,1.062243e-06,2020,SFG
303,100.000000,3.705382e-08,2020,NYY
317,100.000000,4.728690e-09,2020,OAK
326,100.000000,1.420497e-10,2020,CLE


In [29]:
# add top three most likely teams for each season to a dictionary
seasons = probs_df['Year'].unique()
top_three = {}
for season in seasons:
    df = probs_df.loc[probs_df['Year'] == season]
    df = df.sort_values(by='True', ascending=False)
    top_three[season] = {'first_prediction':df.iloc[0,3], 'second_prediction': df.iloc[1,3], 'third_prediction': df.iloc[2,3]}
    

In [30]:
# put dictionary into dataframe
top_three_df = pd.DataFrame(top_three)
top_three_df = top_three_df.transpose()

In [31]:
# sort by year
top_three_df = top_three_df.sort_index()

In [32]:
# adding year as a column
top_three_df.reset_index(inplace=True)
top_three_df = top_three_df.rename(columns={'index':'Year'})

In [33]:
# top_three_df

In [34]:
# bring in actual winners
even_df = top_three_df.merge(winners_df, on='Year')

In [35]:
# dataframe cleanup
even_df.drop(columns=['Prediction','Actual'], inplace=True)

In [36]:
# even_df.head()

In [37]:
even_df.rename(columns={'Team':'actual'}, inplace=True)

In [38]:
# even_df

# Part 2 - Repeat for odd years

In [39]:
# import dependencies
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [40]:
# read in data
pd.set_option('display.max_columns', None)
mlb_df = pd.read_csv('resources/mlb_data.csv')
# mlb_df.head()

In [41]:
# change the League column to be binary instead of string
for i, row in mlb_df.iterrows():
#     print(i,row['Lg'])
    if row['Lg'] == 'AL':
        mlb_df.at[i,'Lg'] = 0
#         print(f'new value at {i} is {mlb_df.at[i,"Lg"]}')
    elif row['Lg'] == 'NL':
        mlb_df.at[i,'Lg'] = 1
#         print(f'new value at {i} is {mlb_df.at[i,"Lg"]}')
    

In [42]:
# league is now a 1 or 0
# mlb_df.head()
# mlb_df.dtypes

# Split into training and testing set
Manually split the data by season. Even years will be used for training, while odd years will be used for testing

In [43]:
# get X and Y train for making the model
train_df = mlb_df.loc[mlb_df['Year'] % 2 == 0]
train_data = train_df.values
X_train = train_data[:, 3:22]
y_train = train_data[:,23]

In [44]:
# get X and Y for making the model
test_df = mlb_df.loc[mlb_df['Year'] % 2 == 1]
test_data = test_df.values
X_test = test_data[:, 3:22]
y_test = test_data[:,23]

# Scaling and One-hot encoding

In [45]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

In [46]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [47]:
# label encode the winner column
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
label_encoder.fit(y_test)
encoded_y_test = label_encoder.transform(y_test)

In [48]:
# One-hot encoding
from keras.utils import to_categorical

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
# y_train_categorical

# Create model

In [49]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [50]:
X_train.shape

(330, 19)

In [51]:
from tensorflow.keras.layers import Dense
number_inputs = X_train.shape[1]
number_hidden_nodes = 1000
model.add(Dense(units=number_hidden_nodes, activation='relu', input_dim=number_inputs))
model.add(Dense(units=number_hidden_nodes, activation='relu'))

In [52]:
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax'))

In [53]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1000)              20000     
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 2002      
Total params: 1,023,002
Trainable params: 1,023,002
Non-trainable params: 0
_________________________________________________________________


In [54]:
# # Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [55]:
print(X_train_scaled.shape)
print(y_train_categorical.shape)

(330, 19)
(330, 2)


In [56]:
# Fit the model
model.fit(
    x=X_train_scaled,
    y=y_train_categorical,
    epochs=200,
    shuffle=True,
    verbose=0
)

In [57]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

300/300 - 0s - loss: 0.6767 - acc: 0.9633
Loss: 0.6767239638774967, Accuracy: 0.9633333086967468


In [58]:
encoded_predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
# print(f"Predicted classes: {prediction_labels}")
# print(f"Actual Labels: {list(y_test)}")

# Post-process and analyze results

In [59]:
pd.set_option('display.max_rows', None)
predictions_df = pd.DataFrame({'Prediction':prediction_labels, 'Actual':list(y_test)})
# predictions_df.head()

In [60]:
winners_df = predictions_df.loc[predictions_df['Actual']]
# winners_df

In [61]:
# connect predictions to the team and season using iloc and test_df
teams = []
years = []
for i, row in winners_df.iterrows():
    team = test_df.iloc[i,2]
    year = test_df.iloc[i,1]
    teams.append(team)
    years.append(year)

In [62]:
# display predictions by team
winners_df['Team'] = teams
winners_df['Year'] = years
# winners_df

In [63]:
# using probabilities 
probs = model.predict_proba(X_test_scaled)
# probs

In [64]:
# dataframe cleanup
probs_df = pd.DataFrame.from_records(probs)
probs_df[0] = probs_df[0] * 100
probs_df[1] = probs_df[1] * 100
probs_df.rename(columns={0:'False', 1:'True'}, inplace=True)
# probs_df.head()

In [65]:
# connect predictions to the team and season using iloc and test_df
all_teams = []
all_years = []
for i, row in probs_df.iterrows():
    team = test_df.iloc[i,2]
    year = test_df.iloc[i,1]
    all_teams.append(team)
    all_years.append(year)
# print(all_years)

In [66]:
# add teams to dataframe
probs_df['Year'] = all_years
probs_df['Team'] = all_teams
probs_df.sort_values(by=['Year','True'],ascending=False)
# probs_df.head()

,False,True,Year,Team
3,99.887985,1.120139e-01,2019,LAD
2,100.000000,1.689557e-08,2019,HOU
5,100.000000,1.699026e-11,2019,OAK
25,100.000000,1.281866e-11,2019,WSN
6,100.000000,1.195530e-11,2019,CHC
8,100.000000,9.354942e-13,2019,ATL
11,100.000000,1.055562e-13,2019,BOS
50,100.000000,7.273552e-14,2019,TBR
13,100.000000,7.084214e-16,2019,NYM
63,100.000000,2.870776e-17,2019,STL


In [67]:
# add top three most likely teams for each season to a dictionary
seasons = probs_df['Year'].unique()
top_three = {}
for season in seasons:
    df = probs_df.loc[probs_df['Year'] == season]
    df = df.sort_values(by='True', ascending=False)
    top_three[season] = {'first_prediction':df.iloc[0,3], 'second_prediction': df.iloc[1,3], 'third_prediction': df.iloc[2,3]}
    

In [68]:
# put dictionary into dataframe
top_three_df = pd.DataFrame(top_three)
top_three_df = top_three_df.transpose()

In [69]:
# sort by year
top_three_df = top_three_df.sort_index()

In [70]:
# adding year as a column
top_three_df.reset_index(inplace=True)
top_three_df = top_three_df.rename(columns={'index':'Year'})

In [71]:
# top_three_df

In [72]:
# bring in actual winners
odd_df = top_three_df.merge(winners_df, on='Year')

In [73]:
# dataframe cleanup
odd_df.drop(columns=['Prediction','Actual'], inplace=True)

In [74]:
odd_df.head()

,Year,first_prediction,second_prediction,third_prediction,Team
0,2001,SEA,CHC,OAK,ARI
1,2003,HOU,SFG,BOS,FLA
2,2005,STL,BOS,ATL,CHW
3,2007,SDP,ATL,STL,BOS
4,2009,NYY,BOS,PHI,NYY


In [75]:
odd_df.rename(columns={'Team':'actual'}, inplace=True)

In [76]:
# odd_df.head()

In [77]:
# even_df.head()

In [78]:
df = pd.concat([even_df, odd_df])
# df.head()

In [79]:
df = df.sort_values(by='Year')
df.reset_index(inplace=True, drop=True)
df.head()

,Year,first_prediction,second_prediction,third_prediction,actual
0,2000,ATL,ARI,NYM,NYY
1,2001,SEA,CHC,OAK,ARI
2,2002,NYY,ARI,BOS,ANA
3,2003,HOU,SFG,BOS,FLA
4,2004,STL,HOU,BOS,BOS


In [80]:
# add full names to df
teams =  ['ANA','ARI','ATL','BAL','BOS','CHC','CHW','CIN','CLE','COL','DET','FLA','HOU','KCR','LAA','LAD','MIA','MIL','MIN','MON','NYM','NYY','OAK','PHI','PIT','SDP','SEA','SFG','STL','TBD','TBR','TEX','TOR','WSN']
full_names = ['Anahiem Angels','Arizona Diamondbacks','Atlanta Braves','Baltimore Orioles','Boston Red Sox','Chicago Cubs','Chicago White Sox','Cincinnati Reds','Cleveland Indians','Colorado Rockies','Detroit Tigers','Florida Marlins','Houston Astros','Kansas City Royals','Los Angeles Angels','Los Angeles Dodgers','Miami Marlins','Milwaukee Brewers','Minnesota Twins','Montreal Expos','New York Mets','New York Yankees','Oakland Athletics','Philadelphia Phillies','Pittsburgh Pirates','San Diego Padres','Seattle Mariners','San Francisco Giants','St. Louis Cardinals','Tampa Bay Devil Rays','Tampa Bay Rays','Texas Rangers','Toronto Blue Jays','Washington Nationals']
full_names_df = pd.DataFrame({'abbr':teams,'fn':full_names})
# full_names_df

In [81]:
df = df.merge(full_names_df, right_on='abbr', left_on='actual')

In [82]:
df.rename(columns={'fn':'actual_fn'},inplace=True)
df.drop(columns='abbr',inplace=True)

In [83]:
df = df.merge(full_names_df, right_on='abbr', left_on='first_prediction')

In [84]:
df.rename(columns={'fn':'first_prediction_fn'},inplace=True)
df.drop(columns='abbr',inplace=True)

In [85]:
df = df.merge(full_names_df, right_on='abbr', left_on='second_prediction')

In [86]:
df.rename(columns={'fn':'second_prediction_fn'},inplace=True)
df.drop(columns='abbr',inplace=True)

In [87]:
df = df.merge(full_names_df, right_on='abbr', left_on='third_prediction')

In [88]:
df.rename(columns={'fn':'third_prediction_fn'},inplace=True)
df.drop(columns='abbr',inplace=True)

In [89]:
df = df.sort_values('Year')

In [90]:
df

,Year,first_prediction,second_prediction,third_prediction,actual,actual_fn,first_prediction_fn,second_prediction_fn,third_prediction_fn
0,2000,ATL,ARI,NYM,NYY,New York Yankees,Atlanta Braves,Arizona Diamondbacks,New York Mets
9,2001,SEA,CHC,OAK,ARI,Arizona Diamondbacks,Seattle Mariners,Chicago Cubs,Oakland Athletics
1,2002,NYY,ARI,BOS,ANA,Anahiem Angels,New York Yankees,Arizona Diamondbacks,Boston Red Sox
2,2003,HOU,SFG,BOS,FLA,Florida Marlins,Houston Astros,San Francisco Giants,Boston Red Sox
3,2004,STL,HOU,BOS,BOS,Boston Red Sox,St. Louis Cardinals,Houston Astros,Boston Red Sox
6,2005,STL,BOS,ATL,CHW,Chicago White Sox,St. Louis Cardinals,Boston Red Sox,Atlanta Braves
12,2006,NYM,MIN,LAA,STL,St. Louis Cardinals,New York Mets,Minnesota Twins,Los Angeles Angels
16,2007,SDP,ATL,STL,BOS,Boston Red Sox,San Diego Padres,Atlanta Braves,St. Louis Cardinals
11,2008,BOS,CHC,LAA,PHI,Philadelphia Phillies,Boston Red Sox,Chicago Cubs,Los Angeles Angels
5,2009,NYY,BOS,PHI,NYY,New York Yankees,New York Yankees,Boston Red Sox,Philadelphia Phillies


In [91]:
df.to_json('nn_predictions.json')